In [1]:
import random
def isComplete(assignment):
    return None not in (assignment.values())

def select_unassigned_variable(variables, assignment):
    for var in variables:
        if assignment[var] is None:
            return var

def is_consistent(assignment, constraints):
    for constraint_violated in constraints:
        if constraint_violated(assignment):
            return False
    return True

def init_assignment(csp):
    assignment = {}
    for var in csp["VARIABLES"]:
        assignment[var] = None
    return assignment

def add_constraint(csp, constraint): 
    csp['CONSTRAINTS'].append(constraint)
    
def recursive_backtracking(assignment, csp):
    if isComplete(assignment):
        return assignment
    var = select_unassigned_variable(csp["VARIABLES"], assignment)
    for value in csp["DOMAINS"]:
        assignment[var] = value
        if is_consistent(assignment, csp["CONSTRAINTS"]):
            result = recursive_backtracking(assignment, csp)
            if result != "FAILURE":
                return result
        assignment[var] = None
    return "FAILURE"

def binary_constraint(var_pair, violations):
    (v1,v2) = var_pair
    return lambda asmt: (asmt[v1], asmt[v2]) in violations


### 1
def init_csp_aus_coloring():
    csp = {}
    csp["VARIABLES"] = ["WA", "NT", "SA", "NSW", "Q", "V", "T"]
    csp['CONSTRAINTS'] = []
    aus_violations = [('red', 'red'), ('blue', 'blue'), ('green', 'green')]
    aus_var_pairs = [("WA", "NT"), ("WA", "SA") , ("SA", "NT") , ("SA", "Q") , ("NT", "Q") , ("SA","NSW"), ("SA", "V"), ("Q", "NSW") , ("V" , "NSW")]
    for var_pair in aus_var_pairs:
        add_constraint(csp, binary_constraint(var_pair, violations=aus_violations))
    csp["DOMAINS"] = ['red', 'blue', 'green']
    return csp

csp = init_csp_aus_coloring()
assignment = init_assignment(csp)
recursive_backtracking(assignment, csp)
print("Australia Coloring: ", assignment)

### 2
csp = init_csp_aus_coloring()
assignment = init_assignment(csp)
assignment['WA'] = 'blue'
recursive_backtracking(assignment, csp)
print("WA is blue: ", assignment)

### 3
def unary_constraint(var_v1, violations):
    return lambda asmt: asmt[var_v1] in violations

csp = init_csp_aus_coloring()
add_constraint(csp, unary_constraint(var_v1='WA', violations=['blue']))
add_constraint(csp, unary_constraint(var_v1='T', violations=['blue']))
assignment = init_assignment(csp)
recursive_backtracking(assignment, csp)
print("Unary Constraint: ", assignment)

### 4

### 5
def is_consistent(assignment, constraints):
    for constraint_violated in constraints:
        if constraint_violated(assignment) is not None:
            return False
    return True

def init_random_assignment(csp):
    assignment = {}
    for var in csp["VARIABLES"]:
        assignment[var] = random.choice(csp['DOMAINS'])
    return assignment

def binary_constraint(var_pair, violations):
    (v1,v2) = var_pair
    return lambda asmt: (lambda var:  var == v1 or var == v2) if (asmt[v1], asmt[v2]) in violations else None

def count_conflicts(variables, assignment, constraints):
    conflicts = {var : 0 for var in variables}
    for var in variables:
        for constraint_violated in constraints:
            var_involved = constraint_violated(assignment)
            if var_involved is not None:
                conflicts[var] += 1 if var_involved(var) else 0
    return conflicts

def select_random_conflicted_variable(csp, assignment):
    variables = csp['VARIABLES']
    var_conflicts = count_conflicts(variables, assignment, csp['CONSTRAINTS'])
    var = random.choice(variables)
    while var_conflicts[var] == 0:
        var = random.choice(variables)
    return var

def select_minimun_conflicts(csp, assignment, var):
    value_conflicts = []
    for value in csp['DOMAINS']:
        assignment[var] = value
        conflicts = count_conflicts(csp['VARIABLES'], assignment, csp['CONSTRAINTS'])
        value_conflicts.append(sum(conflicts.values()))
    best_index = value_conflicts.index(min(value_conflicts))
    return csp['DOMAINS'][best_index]

def min_conflicts(csp, max_steps=1000):
    assignment = init_random_assignment(csp)
    for i in range(max_steps):
        if is_consistent(assignment, csp['CONSTRAINTS']):
            return assignment
        var = select_random_conflicted_variable(csp, assignment)
        value = select_minimun_conflicts(csp, assignment, var)
        assignment[var] = value
    return "FAILURE"

## TODO BUG check if binary constraint is changed
## TODO check for different n variables
## TODO Integrate with previous part for points...
csp = init_csp_aus_coloring() 
min_conflicts(csp)
print("Min Conflicts: ", assignment)

Australia Coloring:  {'WA': 'red', 'NT': 'blue', 'SA': 'green', 'NSW': 'blue', 'Q': 'red', 'V': 'red', 'T': 'red'}
WA is blue:  {'WA': 'blue', 'NT': 'red', 'SA': 'green', 'NSW': 'red', 'Q': 'blue', 'V': 'blue', 'T': 'red'}
Unary Constraint:  {'WA': 'red', 'NT': 'blue', 'SA': 'green', 'NSW': 'blue', 'Q': 'red', 'V': 'red', 'T': 'red'}
Min Conflicts:  {'WA': 'red', 'NT': 'blue', 'SA': 'green', 'NSW': 'blue', 'Q': 'red', 'V': 'red', 'T': 'red'}
